In [26]:
# import packages to read sqlite file
import sqlite3
import pandas as pd
import time  
import datetime
import calendar
import argparse

# import packages to write a json file
import os
import json

- Convert date string (HHMMSS.MS) to PTG timestamp format (epoch)

In [92]:
# Convert date string (HHMMSS) to PTG timestamp format (epoch)
def  utc_hms_date_to_epoch(utc_date_str):
    # utc_date_str = '17:29:43.005'
    datetime_value = datetime.datetime.strptime(utc_date_str, '%H:%M:%S.%f')
    dt_now = datetime.datetime.now()
    # set year, month, day of the datetime object to current date's year, month and day via replace
    # since there is no Year, Month, Day info (this info is required to computed a positive epoch value. otherwise it will be negative (1900-01-01))
    dt = datetime_value.replace(year=dt_now.year, month=dt_now.month, day=dt_now.day) # replace with current YYMMDD info
    epoch_format = calendar.timegm(dt.timetuple())
    ptg_timestamp_format = str(epoch_format*1000)+'-0'
    return ptg_timestamp_format    

In [93]:
utc_hms_date_to_epoch("17:29:43.005")

1900-01-01 17:29:43.005000


'1684430983000-0'

Create a database connection to the SQLite database and return this as a dataframe

In [28]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file, uri=True)
    except Exception as e:
        print(e)
    return conn

def get_dataframe_from_sqlite(db_file, table_name):
    # fancy read-only connection
    conn = create_connection(db_file)
    # create the dataframe from a query
    query = "SELECT * FROM " + table_name
    df = pd.read_sql_query(query, conn)
    return df

#### Getting actions from a dataframe

In [68]:
def get_actions_json(df):
    json_values = []
    unique_actions = df["Event"].unique()
    
    df2 = df[df['timestamp'].notnull()] # remove null timestamps
    df3 = df2[pd.to_datetime(df2['timestamp'], errors='coerce',format='%H:%M:%S.%f').notnull()] # check correct format
    uniqueTs = df3['timestamp'].unique()
    for ts in uniqueTs:
        detected_actions_per_ts = df[df['timestamp'] == ts]['Event'].values
        row_dic = {"timestamp": utc_hms_date_to_epoch(ts) } # get time from sqlite file
        for action in unique_actions:
            if(action in list(detected_actions_per_ts)):
                row_dic[action] = 1
            else:
                row_dic[action] = 0
        json_values.append(row_dic)        
        sorted_json_data = sorted(json_values, key=lambda d: d['timestamp'].split('-')[0])     
    return sorted_json_data

### Getting actions from a SQLite database.


In [69]:
filename = 'egovlp_action_steps_v10.json' # 'detic-image-fixed-labels.json'
isFile = os.path.isfile(filename)

df_table = get_dataframe_from_sqlite("0293_13.sqlite", "ocarina_mission_log")

json_data = get_actions_json(df_table)

if (not isFile):
    with open(filename, 'w') as fp:
        json.dump(json_data, fp, indent=4)
else:
    print(f'An error occurred writing to {filename}.') 

#### Getting steps from a dataframe


In [72]:
# {
#   "step_id": 0,
#   "step_status": "NEW",
#   "step_description": "Place tortilla on cutting board.",
#   "error_status": false,
#   "error_description": "",
#   "timestamp": "1679339146083-0"
# }

def get_steps_json(df):
    json_values = []
    
    df2 = df[df['timestamp'].notnull()] # remove null timestamps
    df3 = df2[pd.to_datetime(df2['timestamp'], errors='coerce',format='%H:%M:%S.%f').notnull()] # check correct format
    uniqueTs = df3['timestamp'].unique()
    for ts in uniqueTs:
        detected_steps_per_ts = df[df['timestamp'] == ts]['Step'].values
        step = list(detected_steps_per_ts)[0]
        timestamp_value = utc_hms_date_to_epoch(ts)# get time from sqlite file
        row_dic = {"step_id": step,
                    "step_status": "NEW",
                    "step_description": "",
                    "error_status": False,
                    "error_description": "",
                    "timestamp": timestamp_value
                   }
        json_values.append(row_dic)        
        sorted_json_data = sorted(json_values, key=lambda d: d['timestamp'].split('-')[0])     
    return sorted_json_data

### Getting steps from a SQLite database.

In [73]:
filename = 'reasoning_check_status_v10.json' # 'detic-image-fixed-labels.json'
isFile = os.path.isfile(filename)

df_table = get_dataframe_from_sqlite("0293_13.sqlite", "ocarina_mission_log")

json_data = get_steps_json(df_table)

if (not isFile):
    with open(filename, 'w') as fp:
        json.dump(json_data, fp, indent=4)
else:
    print(f'An error occurred writing to {filename}.') 

### Generating metadata from a objects json file and computing video duration.

In [88]:
import argparse
import json
import os

# required to install "pip install moviepy"

from moviepy.editor import VideoFileClip

def getMetadata(video_path, objects_file_path):
    clip = VideoFileClip(video_path)
    # Opening JSON file that contains objects (it must be sorted by timestamps)
    file_objects = open(objects_file_path)
    # returns JSON object as a dictionary
    data = json.load(file_objects)

    metadata ={
            "duration_secs": int(clip.duration),
            "first-entry": data[0]["timestamp"],
            "last-entry": data[len(data)-1]["timestamp"],
        }
    return metadata


In [90]:

video_path = "ngc_0293_13.mp4"
objects_file_path = 'objects_from_sql_v9.json'
filename = 'ngc_0293_13_additional_metadata_v1.json' # 'detic-image-fixed-labels.json'

isFile = os.path.isfile(filename)
json_data = getMetadata(video_path, objects_file_path)

if (not isFile):
    with open(filename, 'w') as fp:
        json.dump(json_data, fp, indent=4)
else:
    print(f'An error occurred writing to {filename}.') 